In [1]:
import os, time, sys
from vllm import LLM
sys.path.append('../elyra_pipeline/')
from postgres_utilities import connect_db, close_db, execute_sql_results, execute_sql_results_np, execute_sql

INFO 05-05 14:46:36 [__init__.py:239] Automatically detected platform cuda.


In [2]:
embedding_endpoint =  os.environ.get('embedding_model_endpoint')
model_name = 'gritlm-7b'
number_data_rows = 1000

In [3]:
embedding_endpoint = f"{embedding_endpoint}/v1/embeddings"

In [4]:
delete_model = f"select aidb.delete_model('{model_name}');"
create_model = f"""
    select aidb.create_model(
    '{model_name}',
    'embeddings',
    '{{"model":"{model_name}", "url":"{embedding_endpoint}", 
    "dimensions":4096}}'::JSONB,
    '{{"api_key":""}}'::JSONB, true);
    """
generate_embeddings = f"select aidb.encode_text('{model_name}', productdisplayname) from products limit {number_data_rows}"
#print(delete_model)
#print(create_model)
#print(generate_embeddings)

In [5]:
try:
    conn = connect_db()
    execute_sql(conn, delete_model)
    execute_sql(conn, create_model)

    test_description = "Starting online embedding test using edb with OpenShift AI model"
    print(test_description)
    print(len(test_description) * "-") 
    tic = time.perf_counter()
    print("Embeddings computed ...", end=" ")

    results = execute_sql_results_np(conn, generate_embeddings)

    assert len(results) == number_data_rows
    toc = time.perf_counter()
    print(f"{number_data_rows}")
    print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
    print(len(test_description) * "-")
except Exception as e:
    if conn is not None and not conn.closed:
        conn.rollback()
    err_msg = f"An error occurred and the transaction was rolled back: {e}"
    raise Exception(err_msg)
finally:
    close_db(conn)

Starting online embedding test using edb with OpenShift AI model
----------------------------------------------------------------
Embeddings computed ... 1000
Time to compute 1000 embeddings: 62.0088 seconds
----------------------------------------------------------------
